In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from algorithm import (
                        NoRegularizationTrainer,
                        GradientTrainer,
                        EGTrainer,
                        CSVDataLoader,
                        MultiTaskModel,
                        MultiTaskDataset,
                        mlt_train_test_split,
                        # true_values_from_data_loader,
                        unique_value_counts,
                        Cindex,
                        brier_score,
                        )
import easydict
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [7]:
data = pd.read_csv('Dataset/processdata.csv', encoding='latin-1')
date_columns = ['Date.of.Last.Contact', 'Date.of.Diagnostic']
data[date_columns] = data[date_columns].apply(pd.to_datetime, errors='coerce')
data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
data.loc[:, 'Survival_Time'] = data['Survival_Time'].replace({-1: 0})
data['indicater'] = np.where(data['Date.of.Death'].isna(), 0, 1)


columns_to_drop = ['Date.of.Death', 'Date.of.Last.Contact', 'Date.of.Diagnostic']
data.drop(columns=columns_to_drop, inplace=True)

train_indices, test_indices = train_test_split(
        range(len(data)), stratify=data['indicater'], random_state=1, test_size=0.25
    )

X = data.drop(['Survival_Time', 'indicater'], axis=1)
time_all = data['Survival_Time'].values
event_all = data['indicater'].values

X_train = X.iloc[train_indices].copy()
X_test = X.iloc[test_indices].copy()
time_train = time_all[train_indices].copy()
time_test = time_all[test_indices].copy()
event_train = event_all[train_indices].copy()
event_test = event_all[test_indices].copy()


columns_to_one_hot = ['RCBP.Name', 'Raca.Color', 'State.Civil', 'Code.Profession', 'Name.Occupation',
                              'Status.Address',
                              'City.Address', 'Description.of.Topography', 'Topography.Code', 'Morphology.Description',
                              'Code.of.Morphology', 'Description.of.Disease', 'Illness.Code', 'Diagnostic.means',
                              'Extension',
                              'Type.of.Death']

for column in columns_to_one_hot:
    top_9_values = X_train[column].value_counts().nlargest(9).index
    X_train[column] = X_train[column].where(X_train[column].isin(top_9_values), 'other')
    X_test[column] = X_test[column].where(X_test[column].isin(top_9_values), 'other')


X_train = pd.get_dummies(X_train, columns=columns_to_one_hot)
X_test = pd.get_dummies(X_test, columns=columns_to_one_hot)

print((X_test.columns == X_train.columns).all())

columns_to_binarize = ['Gender', 'Indicator.of.Rare.Case']    
for column in columns_to_binarize:
    lb = LabelBinarizer()
    X_train[column] = lb.fit_transform(X_train[column])
    X_test[column] = lb.transform(X_test[column])

scaler = MinMaxScaler()
X_train['Age'] = scaler.fit_transform(X_train[['Age']])
X_test['Age'] = scaler.transform(X_test[['Age']])

Tmax = time_train.max()
num_intervals=7
intervals = [(i * (Tmax // num_intervals), (i + 1) * (Tmax // num_intervals)) for i in range(num_intervals)]

Y_train = np.zeros((len(time_train), num_intervals), dtype=np.int_)
# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_train):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y_train[i, j] = 1
Y_train = torch.Tensor(Y_train)


Y_test = np.zeros((len(time_test), num_intervals), dtype=np.int_)
# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_test):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y_test[i, j] = 1
Y_test = torch.Tensor(Y_test)



# Creat mask matrix
W_train = np.zeros((len(time_train), num_intervals), dtype=np.int_)
# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_train, event_train)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W_train[i, j] = 0
        else:
            W_train[i, j] = 1
W_train = torch.Tensor(W_train)



# Creat mask matrix
W_test = np.zeros((len(time_test), num_intervals), dtype=np.int_)
# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_test, event_test)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W_test[i, j] = 0
        else:
            W_test[i, j] = 1
W_test = torch.Tensor(W_test)


for col in X_train.columns:
    if X_train[col].dtype == bool:
        X_train[col] = X_train[col].astype(int)
X_train = torch.tensor(X_train.values, dtype=torch.float32)

for col in X_test.columns:
    if X_test[col].dtype == bool:
        X_test[col] = X_test[col].astype(int)
X_test = torch.tensor(X_test.values, dtype=torch.float32)

# Y_train = Y[train_indices]
# Y_test = Y[test_indices]
# W_train = W[train_indices]
# W_test  = W[test_indices]
Y_train_transform = [Y_train[:, i:i + 1] for i in range(Y_train.size(1))]
Y_test_transform = [Y_test[:, i:i + 1] for i in range(Y_test.size(1))]
W_train_transform = [W_train[:, i:i+1] for i in range(W_train.size(1))]
W_test_transform = [W_test[:, i:i+1] for i in range(W_test.size(1))]

args = easydict.EasyDict({
    "batch_size": 64,
    "lr": 0.01,
    "epochs": 200,
    "clip": 5.0,
    "lambda_reg": 0.01,
    "save_path": "outputfiles",
    "eg_k" : 1, 
    "early_stop_patience":15,
})

train_dataset = MultiTaskDataset(X_train, Y_train_transform, W_train_transform, event_train)
test_dataset = MultiTaskDataset(X_test, Y_test_transform, W_test_transform, event_test)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)

model = MultiTaskModel(X_train.shape[1], Y_train.shape[1])


trainer = NoRegularizationTrainer(model,train_loader,test_loader, args)
trainer.train()

True


  0%|          | 1/200 [00:01<05:14,  1.58s/it]

End of Epoch 0, Average Training Loss: 1.4687, Average Gradient Norm: 1.9357
End of Epoch 0, Average Validation Loss: 1.4798
Current Learning Rate: 0.010000


  1%|          | 2/200 [00:03<05:01,  1.52s/it]

End of Epoch 1, Average Training Loss: 1.1020, Average Gradient Norm: 1.6548
End of Epoch 1, Average Validation Loss: 1.0778
Current Learning Rate: 0.010000


...

End of Epoch 21, Average Training Loss: 0.9026, Average Gradient Norm: 1.9630
End of Epoch 21, Average Validation Loss: 1.0966
Current Learning Rate: 0.000100
End of Epoch 22, Average Training Loss: 0.8902, Average Gradient Norm: 2.0069


 12%|█▏        | 23/200 [00:32<04:22,  1.48s/it]

End of Epoch 22, Average Validation Loss: 1.0972
Current Learning Rate: 0.000100
End of Epoch 23, Average Training Loss: 0.8972, Average Gradient Norm: 1.9687


 12%|█▏        | 23/200 [00:34<04:24,  1.50s/it]

End of Epoch 23, Average Validation Loss: 1.0952
Current Learning Rate: 0.000100
Early stopping triggered after 24 epochs.


In [8]:
class CindexOptimized(torch.nn.Module):

    def __init__(self):

        super(CindexOptimized, self).__init__()

    def forward(self, y, y_hat, status):
        if not torch.is_tensor(y):
            y = torch.tensor(y, dtype=torch.float32)
        if not torch.is_tensor(y_hat):
            y_hat = torch.tensor(y_hat, dtype=torch.float32)
        if not torch.is_tensor(status):
            status = torch.tensor(status, dtype=torch.float32)

        # replacing loop acceleration with matrix calculation

        y_diff = y.unsqueeze(1) - y.unsqueeze(0)
        y_hat_diff = y_hat.unsqueeze(1) - y_hat.unsqueeze(0)
        # status[i] and status[j] mark whether to censored data
        status_i = status.unsqueeze(1)
        status_j = status.unsqueeze(0)
        valid_pairs = torch.logical_or((y_diff <= 0) & (status_i == 1), (y_diff >= 0) & (status_j == 1)).float()
        torch.diagonal(valid_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = torch.logical_or((y_diff <= 0) & (y_hat_diff <= 0)&(status_i == 1),(y_diff >= 0) & (y_hat_diff >= 0)& (status_j == 1)).float()
        torch.diagonal(concordant_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = concordant_pairs.float()
        c_index = concordant_pairs.sum() / valid_pairs.sum()
        return c_index.item()



cindex_calculator_optimized = CindexOptimized()

trainer.load_best_checkpoint()
predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
c11_train = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Training Data: {c11_train:.4f}")


predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
c11_test = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Test Data: {c11_test:.4f}")

Loaded model from checkpoint at epoch 9 with best validation loss 0.9998
C-index for Training Data: 0.9791
C-index for Test Data: 0.9779


In [9]:
Y = np.zeros((len(time_all), num_intervals), dtype=np.int_)
# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_all):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y[i, j] = 1
Y = torch.Tensor(Y)

In [10]:
trainer.load_best_checkpoint()

print("Evaluation On Train Data \n")

predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
print("Y_true Train")
unique_value_counts(Y_true)
print("Y_hat Train")
unique_value_counts(Y_hat)

cindex_calculator = Cindex()
cindex_test = cindex_calculator(Y_true, Y_hat, events)
bscore_test = brier_score(event_all, Y, events, Y_true, predictions)
print(f"Train Data C-Index = {cindex_test},  BScore = {bscore_test}")



print("\nEvaluation On Test Data \n")

predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
print("Y_true Test")
unique_value_counts(Y_true)
print("Y_hat Test")
unique_value_counts(Y_hat)

cindex_calculator = Cindex()
cindex_test = cindex_calculator(Y_true, Y_hat, events)
bscore_test = brier_score(event_all, Y, events, Y_true, predictions)
print(f"Test Data C-Index = {cindex_test},  BScore = {bscore_test}")

Loaded model from checkpoint at epoch 9 with best validation loss 0.9998
Evaluation On Train Data 
Y_true Train
Unique Values: tensor([0., 1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64)
Counts: tensor([3064, 3004, 1193, 1548, 1232,  714,  200,   53])
Y_hat Train
Unique Values: tensor([0, 1, 2, 3, 4, 5, 6, 7], dtype=torch.int32)
Counts: tensor([2672, 2415, 1061, 1071,  488,  267,    1, 3033])


100%|██████████| 11008/11008 [00:01<00:00, 7446.75it/s]


Train Data C-Index = 0.9790862202644348,  BScore = 0.06266008509244994

Evaluation On Test Data 

Y_true Test
Unique Values: tensor([0., 1., 2., 3., 4., 5., 6., 7.], dtype=torch.float64)
Counts: tensor([1026, 1004,  395,  496,  410,  223,   74,   20])
Y_hat Test
Unique Values: tensor([0, 1, 2, 3, 4, 5, 7], dtype=torch.int32)
Counts: tensor([ 890,  766,  381,  353,  174,   84, 1000])


100%|██████████| 3648/3648 [00:00<00:00, 7982.49it/s]

Test Data C-Index = 0.9779160618782043,  BScore = 0.06779034798891784


In [11]:
import numpy as np
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import integrated_brier_score

Y1 = Y.sum(axis=1).cpu().numpy()
dtype = [('cens', bool), ('time', int)]
Ye = np.array(list(zip(event_all.astype(bool), Y1)), dtype=dtype)
y_train, y_test = Y_train.sum(axis=1).cpu().numpy(), Y_test.sum(axis=1).cpu().numpy()

y_train = np.array(list(zip(event_train.astype(bool), y_train)), dtype=dtype)
y_test =np.array(list(zip(event_test.astype(bool), y_test)), dtype=dtype)

rsf = RandomSurvivalForest(max_depth=2, random_state=1)
rsf.fit(X_train, y_train)
print(f"Random Forest C-Index : {rsf.score(X_test, y_test)}")
survs = rsf.predict_survival_function(X_test)
_times = np.arange(1, 7)
preds = np.asarray([[fn(t) for t in _times] for fn in survs])
print(f"Random Forest Integrated Brier Score : {integrated_brier_score(Ye, y_test, preds, _times)}")
print(f"Random Forest C-Index : {rsf.score(X_train, y_train)}")
survs = rsf.predict_survival_function(X_train)
_times = np.arange(1, 7)
preds = np.asarray([[fn(t) for t in _times] for fn in survs])
print(f"Random Forest Integrated Brier Score : {integrated_brier_score(Ye, y_train, preds, _times)}")


Random Forest C-Index : 0.9458985643111396
Random Forest Integrated Brier Score : 0.14854045547856617
Random Forest C-Index : 0.9449668740302326
Random Forest Integrated Brier Score : 0.1450701356881438
